This study addresses the prevalent issue of greenwashing within the Environmental, Social, and Governance (ESG) reports of German DAX companies. Greenwashing—defined as the practice of misrepresenting sustainability efforts to appear more environmentally friendly than reality—poses significant challenges for stakeholders, including investors, regulators, and the public. These misrepresented reports lead to misinformed decisions that can have widespread economic and environmental consequences. This project utilizes Natural Language Processing (NLP) techniques to analyze approximately 11,000 English-language ESG documents and auxiliary third-party media content. By employing advanced methods such as sentiment analysis, text embeddings, and large language models (LLMs), the study aims to detect inconsistencies and exaggerated sustainability claims effectively. The objective is to enhance transparency in corporate sustainability reports and provide stakeholders with a tool to assess the authenticity of corporate environmental claims. This initiative is crucial for ensuring that ESG data accurately reflect companies' actual sustainability practices, thereby supporting informed decision-making and promoting genuine corporate responsibility. The expected outcomes include a deeper understanding of greenwashing practices and improved methodologies for evaluating corporate sustainability reports. 

In [1]:
#!pip install sentence_transformers

In [2]:
# Make sure that all dependencies are installed.
import pandas as pd
import os
import numpy as np
import sentence_transformers
import torch
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'sentence_transformers'

## 1. Loading the data

In [ ]:
esg_documents_df = pd.read_csv("/kaggle/input/dax-esg-media-dataset/esg_documents_for_dax_companies.csv", sep="|")
esg_documents_df.head()

In [ ]:
# form texts by concatenating title and content
esg_texts = esg_documents_df.apply(lambda row: " ".join([str(row["title"]), str(row["content"])]), axis=1)
esg_texts[0][:100]

In [ ]:
sdg_df = pd.read_csv("/kaggle/input/dax-esg-media-dataset/sdg_descriptions_with_targetsText.csv")
sdg_df.head()

In [ ]:
sdg_texts = sdg_df.apply(lambda row: " ".join([row["name"], row["description"], row["targets"], row["progress"]]), axis=1)
sdg_texts[0][:100]

In [ ]:
companies = sorted(esg_documents_df.company.unique())
companies

## 2. Build embeddings

MPNet base model loaded here is designed to capture a broad understanding of language from diverse datasets, making it suitable for ESG and SDG texts that may contain varied and specialized terminology.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = sentence_transformers.SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

In [ ]:
company_text_embeddings = retriever.encode(esg_texts)
sdg_embeddings = retriever.encode(sdg_texts)

## 3. SDG alignment of the DAX companies

* We model SDG alignment as similarity between the company-related texts and the SDG descriptions. In this section, we first define the similarity function using standard cosine similarity. Then, we demonstrate some possible alignment analyses, their visualisations and interpretations. Finally, we provide suggestions for further, more advanced alignment analyses.

In [ ]:
def cosine_similarity(embedding1, embedding2):
    sim = np.dot(embedding1, embedding2)/(np.linalg.norm(embedding1)*np.linalg.norm(embedding2))
    return sim

### Most relevant SDGs for DAX companies

Let's first consider the overall relevance of the 17 SDGs for all DAX companies taken together. We first compute an embedding that averages over all company texts. Then, we compare this embedding with the embeddings of the SDGs.
Instead of using a simple mean to combine company text embeddings, considered weighting the texts by their length or importance. This will give more relevant documents a greater influence on the resulting average embedding

In [ ]:
# Calculate the length of each document
text_lengths = np.array([len(text) for text in esg_texts])
# Normalize the lengths to sum to 1
weights = text_lengths / text_lengths.sum()
print(weights.sum())

In [ ]:
all_companies_embedding = np.average(company_text_embeddings, axis=0, weights=weights)

In [ ]:
sdg_relevance_scores = [cosine_similarity(all_companies_embedding, sdg_embedding) for sdg_embedding in sdg_embeddings]
sdg_relevance_series = pd.Series(sdg_relevance_scores, index=sdg_df["name"])
sdg_relevance_series.sort_values(inplace=True)
sdg_relevance_series.head()

cosine similarity scores highlight which SDGs are most discussed in company reports, reflecting alignment with SDG language rather than actual efforts or impact. Lower scores for SDGs like Gender Equality suggest underrepresentation in communications, meriting further investigation into potential neglect or underreporting. Conversely, higher scores for SDGs like Zero Hunger indicate a stronger emphasis in companies' sustainability strategies and reporting.

In [ ]:
fig = go.Figure(data=[go.Bar(x=sdg_relevance_series, y=[sdg+" " for sdg in sdg_relevance_series.index], orientation='h')])
# Change the bar mode
fig.update_layout(height=600, width=750, title="SDG relevance for DAX index")
fig.update_xaxes(title="Relevance")
fig.show()

# Clustering

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Assuming company_text_embeddings is a 2D numpy array of embeddings
# Decide on the number of clusters you want to form
n_clusters = 5  # for example, though this number should be chosen based on your data

# Initialize the KMeans clustering algorithm
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit the model on your embeddings to perform the clustering
kmeans.fit(company_text_embeddings)

# Get the cluster labels for each company
cluster_labels = kmeans.labels_

# Optionally, attach the cluster labels to your original dataframe for analysis
esg_documents_df['Cluster'] = cluster_labels

# To analyze the clusters, you might want to look at the companies in each cluster
for i in range(n_clusters):
    print(f"Cluster {i}:")
    print(esg_documents_df[esg_documents_df['Cluster'] == i]['company'].values)
    print()

# If you want to see the centroids of each cluster which are the average of the embeddings in the cluster:
centroids = kmeans.cluster_centers_


In [ ]:
esg_documents_df.to_csv('esg_documents_with_clusters.xlsx', index=False)

In [ ]:
import pandas as pd

# Assuming esg_documents_df has a 'Cluster' column with the cluster labels
cluster_membership_table = esg_documents_df[['company', 'Cluster']].copy()
cluster_membership_table.sort_values('Cluster', inplace=True)

# Save the table to a CSV file for easy sharing and viewing
cluster_membership_table.to_csv('cluster_membership_table.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Count the number of companies in each cluster
cluster_counts = esg_documents_df['Cluster'].value_counts()

# Create a bar plot
plt.figure(figsize=(5, 3))
cluster_counts.plot(kind='bar')
plt.title('Number of Companies in Each Cluster')
plt.xlabel('Cluster')
plt.ylabel('Count')
plt.xticks(rotation=0)  # Keep the x labels readable
plt.tight_layout()
plt.savefig('cluster_sizes_bar_plot.png')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer

# Updated function to display common words for each cluster, excluding stop words
def display_common_words(df, cluster_col='Cluster', text_col='content', n_words=10):
    cluster_common_words = {}
    for cluster in sorted(df[cluster_col].unique()):
        # Combine all the text data for each cluster
        text_data = df[df[cluster_col] == cluster][text_col].str.cat(sep=' ')
        # Use CountVectorizer to count word frequencies, excluding stop words
        vectorizer = CountVectorizer(stop_words='english')
        word_count = vectorizer.fit_transform([text_data])
        # Sum up the word counts and get the word names
        words_freq = word_count.toarray().sum(axis=0)
        words = vectorizer.get_feature_names_out()
        # Combine counts and words, sort by count, and get the most common
        freq_dist = zip(words, words_freq)
        most_common_words = sorted(freq_dist, key=lambda x: x[1], reverse=True)[:n_words]
        # Store and print the most common words
        cluster_common_words[cluster] = most_common_words
        print(f"Cluster {cluster}:")
        for word, freq in most_common_words:
            print(f"{word}: {freq}")
        print("\n")
    return cluster_common_words

# Call the function with the dataframe
common_words_by_cluster = display_common_words(esg_documents_df, 'Cluster', 'content')

### Most relevant SDGs for a specific company, contrasting internal and external data

In this analysis, we focus on a specific company which is defined using the COMPANY variable. We look up the "internal" and "external" embeddings for this company, average them and measure their similarity with each of the SDGs.

In [ ]:
COMPANY = "Brenntag"

internal_company_indices = esg_documents_df[(esg_documents_df.company == COMPANY) & (esg_documents_df.internal == 1)].index
internal_company_embedding = np.mean(company_text_embeddings[internal_company_indices], axis=0)

external_company_indices = esg_documents_df[(esg_documents_df.company == COMPANY) & (esg_documents_df.internal == 0)].index
external_company_embedding = np.mean(company_text_embeddings[external_company_indices], axis=0)

company_sdg_relevance_scores = [[cosine_similarity(internal_company_embedding, sdg_embedding),
                                 cosine_similarity(external_company_embedding, sdg_embedding)] for sdg_embedding in sdg_embeddings]
company_sdg_relevance_df = pd.DataFrame.from_records(company_sdg_relevance_scores, index=sdg_df["name"], columns=["internal", "external"])
company_sdg_relevance_df.sort_values("internal", inplace=True)
company_sdg_relevance_df.head()

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Internal', x=company_sdg_relevance_df["internal"], y=company_sdg_relevance_df.index, orientation='h'),
    go.Bar(name='External', x=company_sdg_relevance_df["external"], y=company_sdg_relevance_df.index, orientation='h')
])

fig.update_layout(barmode='group', height=600, width=750, title=f"SDG Relevance for {COMPANY}")
fig.update_xaxes(title="Relevance")
fig.show()

These scores indicate which SDGs are most closely associated with Brenntag's sustainability narrative, from both the company's perspective and that of external media. The higher scores on external metrics for certain goals could suggest a disparity between how the company perceives its ESG efforts and how these efforts are perceived or reported by external parties. This discrepancy is particularly valuable for identifying potential areas of greenwashing or for highlighting the aspects of sustainability that may need more attention from the company.

Next steps to further explore the data: modify the chart to answer the following questions:

- What are the internally most important SDGs for BMW?
- What are the most important SDGs for BMW when we take into account both the internal and the external data?
- What are the SDGs for BMW with the largest relevance gap between internal and external data?

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have already computed 'company_sdg_relevance_scores' as shown in your notebook snippet
COMPANY = 'BMW'
internal_indices = esg_documents_df[(esg_documents_df['company'] == COMPANY) & (esg_documents_df['internal'] == 1)].index
external_indices = esg_documents_df[(esg_documents_df['company'] == COMPANY) & (esg_documents_df['internal'] == 0)].index

# Aggregate internal and external embeddings for BMW
internal_bmw_embedding = np.mean(company_text_embeddings[internal_indices], axis=0)
external_bmw_embedding = np.mean(company_text_embeddings[external_indices], axis=0)

# Assuming 'sdg_embeddings' is a 2D numpy array with embeddings for each SDG
company_sdg_relevance_scores = [
    (
        sdg_df.loc[idx]["name"],  # SDG Name
        cosine_similarity(internal_bmw_embedding, sdg_embeddings[idx]),  # Internal similarity score
        cosine_similarity(external_bmw_embedding, sdg_embeddings[idx])   # External similarity score
    )
    for idx in range(len(sdg_embeddings))  # Assuming the index aligns with the SDGs order
]

# Create DataFrame from relevance scores
company_sdg_relevance_df = pd.DataFrame.from_records(
    company_sdg_relevance_scores,
    columns=['SDG', 'Internal', 'External']
)

# Calculate the relevance gap between internal and external data
company_sdg_relevance_df['Relevance Gap'] = company_sdg_relevance_df['Internal'] - company_sdg_relevance_df['External']

# Internally most important SDGs for BMW
internally_important_sdgs = company_sdg_relevance_df.sort_values('Internal', ascending=False)

# Most important SDGs for BMW considering both internal and external data
# A simple way to estimate overall importance might be to average the internal and external scores
company_sdg_relevance_df['Average Importance'] = company_sdg_relevance_df[['Internal', 'External']].mean(axis=1)
overall_important_sdgs = company_sdg_relevance_df.sort_values('Average Importance', ascending=False)

# SDGs for BMW with the largest relevance gap between internal and external data
sdgs_largest_relevance_gap = company_sdg_relevance_df.sort_values('Relevance Gap', ascending=False)

# Output the top results for each analysis
print("Internally Most Important SDGs for BMW:")
print(internally_important_sdgs.head(), "\n")

print("Most Important SDGs for BMW (Internal and External):")
print(overall_important_sdgs.head(), "\n")

print("SDGs with the Largest Relevance Gap for BMW:")
print(sdgs_largest_relevance_gap.head(), "\n")



•	Internally Most Important SDGs for BMW

1.	Industry, Innovation, and Infrastructure: 
This is highlighted as the most important SDG in BMW's internal documents, suggesting a significant focus on innovation and sustainable infrastructure within their business practices. This emphasis likely reflects BMW's development of electric vehicles and the establishment of eco-friendly production facilities, aligning with industry trends toward greener and more efficient infrastructures.

2.	Affordable and Clean Energy: 
This SDG is also prioritized highly by BMW, reflecting a focus on energy efficiency and clean energy initiatives within their operations. For example, BMW has invested in renewable energy solutions across its manufacturing plants, such as using wind turbines to power facilities like the Leipzig plant in Germany.

3.	Responsible Consumption and Production: 
BMW emphasizes responsible production by implementing sustainable practices such as reducing waste, increasing recycling, and using eco-friendly materials like water-based paints and recycled plastics in their vehicles, underscoring their commitment to minimizing environmental impact.

4.	Sustainable Cities and Communities: 
BMW's internal reports suggest a commitment to supporting sustainable cities and communities, likely through mobility solutions and urban infrastructure projects.

5.	Good Health and Well-being: 
Finally, good health and well-being are considered important, suggesting that BMW values health-related aspects of sustainability, perhaps related to product safety and employee wellness.

•	Most Important SDGs for BMW (Considering Both Internal and External Data)

When both internal and external data are considered, the SDGs that emerge as most important for BMW largely mirror those identified as internally important:

•	Industry, Innovation, and Infrastructure
•	Affordable and Clean Energy
•	Responsible Consumption and Production 
•	Sustainable Cities and Communities 
•	Reduced Inequalities 

These SDGs represent areas where BMW's sustainability reporting and the external perception of its ESG efforts align, indicating a consistent public-facing narrative on these goals.

•	SDGs with the Largest Relevance Gap for BMW

The SDGs with the most substantial relevance gaps between BMW's internal and external narratives are:

1.  Decent Work and Economic Growth: This has the most considerable gap, which suggests external parties has a significantly different perception of BMW's impact on economic growth and employment quality compared to how BMW reports it.

2.  Quality Education: A notable gap in this area indicates that BMW's contributions to education, perhaps through corporate social responsibility initiatives, are not as recognized externally as they are emphasized internally.

3.  Clean Water and Sanitation: The relevance gap suggests that BMW's efforts or impact on water and sanitation are perceived differently externally compared to the company's internal reporting.

4.  Responsible Consumption and Production: Although this is a highly ranked SDG internally, the gap indicates that external perceptions may not fully align with BMW's reported efforts in this area.

These gaps highlight areas where BMW might need to improve their sustainability practices, increase transparency, or adjust their external communication strategies to better align external perceptions with their internal reporting.


In [ ]:
# Replace 'BMW' with 'Porsche'
COMPANY = 'Porsche'

internal_indices = esg_documents_df[(esg_documents_df['company'] == COMPANY) & (esg_documents_df['internal'] == 1)].index
external_indices = esg_documents_df[(esg_documents_df['company'] == COMPANY) & (esg_documents_df['internal'] == 0)].index

internal_porsche_embedding = np.mean(company_text_embeddings[internal_indices], axis=0)
external_porsche_embedding = np.mean(company_text_embeddings[external_indices], axis=0)

porsche_sdg_relevance_scores = [
    (
        sdg_df.loc[idx]["name"],  # SDG Name
        cosine_similarity(internal_porsche_embedding, sdg_embeddings[idx]),  # Internal similarity score
        cosine_similarity(external_porsche_embedding, sdg_embeddings[idx])   # External similarity score
    )
    for idx in range(len(sdg_embeddings))  # Assuming the index aligns with the SDGs order
]

# Create DataFrame from relevance scores
porsche_sdg_relevance_df = pd.DataFrame.from_records(
    porsche_sdg_relevance_scores,
    columns=['SDG', 'Internal', 'External']
)

# Calculate the relevance gap between internal and external data
porsche_sdg_relevance_df['Relevance Gap'] = porsche_sdg_relevance_df['Internal'] - porsche_sdg_relevance_df['External']

# Sort by Internal, Average, and Relevance Gap
internally_important_sdgs = porsche_sdg_relevance_df.sort_values('Internal', ascending=False)
porsche_sdg_relevance_df['Average Importance'] = porsche_sdg_relevance_df[['Internal', 'External']].mean(axis=1)
overall_important_sdgs = porsche_sdg_relevance_df.sort_values('Average Importance', ascending=False)
sdgs_largest_relevance_gap = porsche_sdg_relevance_df.sort_values('Relevance Gap', ascending=False)

# Assuming you want to output or examine the top results:
print("Internally Most Important SDGs for Porsche:")
print(internally_important_sdgs.head(), "\n")

print("Most Important SDGs for Porsche (Internal and External):")
print(overall_important_sdgs.head(), "\n")

print("SDGs with the Largest Relevance Gap for Porsche:")
print(sdgs_largest_relevance_gap.head(), "\n")


### Heatmap of SDG relevance for all companies

In this analyses, we look at the alignment of all companies with the 17 SDGs, creating an overview over the DAX index.

In [ ]:
company_embeddings = []
for company in companies:
    company_indices = esg_documents_df[esg_documents_df.company == company].index
    company_embedding = np.mean(company_text_embeddings[company_indices], axis=0)
    company_embeddings.append(company_embedding)

In [ ]:
company_records = []

for company_embedding in company_embeddings:
    company_record = []
    for sdg_embedding in sdg_embeddings:
        company_record.append(cosine_similarity(company_embedding, sdg_embedding))
    company_records.append(company_record)

In [ ]:
heatmap_array = np.array(company_records)
heatmap_array.shape

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=heatmap_array,
        x=sdg_df["name"].tolist(),
        y=[company + " " for company in companies],
        colorscale='Viridis'))

fig.update_layout(height=1000)

fig.show()

The heatmap also shows variance across companies for each SDG, indicating that different companies may prioritize different SDGs based on their industry, strategy, and operational focus. For instance, if a certain company has a higher alignment with "Life Below Water," it may suggest their business has a direct impact on marine ecosystems, like shipping or resource extraction.

1.  Affordable and Clean Energy (SDG 7): This goal has the highest total relevance score, it is the most emphasized SDG across DAX companies. Companies are likely focusing on energy efficiency, renewable energy, and clean technology initiatives in their sustainability efforts.

2.  Industry, Innovation, and Infrastructure (SDG 9): Coming in second, this indicates a strong focus on building resilient infrastructure, fostering innovation, and supporting industrial development, which is key to economic growth and sustainable development.

3.  Reduced Inequalities (SDG 10): The third most relevant SDG reflects a significant commitment from these companies to reduce income disparities, promote social, economic, and political inclusion, and ensure equal opportunities.

4.  Responsible Consumption and Production (SDG 12) and Life Below Water (SDG 14): These goals also rank high, showing a focus on sustainable resource use, efficient waste management, and marine conservation.

5.  Partnerships for the Goals (SDG 17): This SDG, while not the highest, still scores relatively well, indicating engagement in partnerships to achieve broader sustainability targets.

6.  Gender Equality (SDG 5) and Peace, Justice, and Strong Institutions (SDG 16): These goals have the lowest total relevance scores. This could mean they are not as prominently reported or targeted in the companies' sustainability strategies or perhaps are areas where the companies need to develop stronger initiatives.



Next steps to further explore the data: modify the chart to answer the following questions:

- What are the companies that are most aligned with the SDGs? (sort by sum of rows)
- Which SDGs are most relevant for the DAX index? (sort by sum of columns)

In [ ]:
import numpy as np
import pandas as pd

# Assuming 'company_text_embeddings' are the embeddings for the company contents
# and 'sdg_embeddings' are the embeddings for the concatenated SDG texts

# Compute cosine similarity for each company and each SDG
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Calculate the alignment score with all SDGs for each company
company_alignment_scores = []

for company in esg_documents_df['company'].unique():
    company_indices = esg_documents_df[esg_documents_df['company'] == company].index
    company_embedding = np.mean(company_text_embeddings[company_indices], axis=0)
    
    company_sdg_scores = [
        cosine_similarity(company_embedding, sdg_embedding) for sdg_embedding in sdg_embeddings
    ]
    
    total_alignment_score = np.sum(company_sdg_scores)
    company_alignment_scores.append((company, total_alignment_score))

# Create a DataFrame from the alignment scores
company_alignment_df = pd.DataFrame(company_alignment_scores, columns=['Company', 'Total Alignment Score'])

# Sort the companies by their total alignment score in descending order
sorted_alignment_df = company_alignment_df.sort_values(by='Total Alignment Score', ascending=False)

# Now sorted_alignment_df contains companies sorted by their alignment with the SDGs
# You can print or save this DataFrame to CSV or Excel for further analysis
sorted_alignment_df.to_csv('companies_sorted_by_sdg_alignment.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 15))  # Adjust the figure size as needed
plt.barh(sorted_alignment_df['Company'], sorted_alignment_df['Total Alignment Score'], color='skyblue')
plt.xlabel('Total Alignment Score')
plt.ylabel('Company')
plt.title('Companies by Total Alignment Score')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest score on top
plt.show()


In [ ]:
# Assume 'company_text_embeddings' are available and are a list of embeddings for each company
# And 'sdg_embeddings' is a list of embeddings for each SDG

# Compute cosine similarity scores between each company and each SDG
all_company_scores = np.array([
    [cosine_similarity(company_embedding, sdg_embedding) for sdg_embedding in sdg_embeddings]
    for company_embedding in company_text_embeddings
])

# Sum scores for each SDG across all companies
sdg_relevance_sums = np.sum(all_company_scores, axis=0)

# Create a DataFrame for SDGs and their summed relevance scores
sdg_relevance_df = pd.DataFrame({
    'SDG': sdg_df['name'],  # or however you have the SDG names stored
    'Total Relevance': sdg_relevance_sums
})

# Sort the DataFrame by the 'Total Relevance' column in descending order
sorted_sdgs_by_relevance = sdg_relevance_df.sort_values(by='Total Relevance', ascending=False)

# Reset the index to get a clean DataFrame
sorted_sdgs_by_relevance.reset_index(drop=True, inplace=True)

# Display the sorted DataFrame
sorted_sdgs_by_relevance


The output appears to be a table that lists the Sustainable Development Goals (SDGs) in descending order of their relevance across the companies in the DAX index, based on a summation of the cosine similarity scores between company reports and SDG embeddings.

Here's an interpretation of the output:

1. **Affordable and Clean Energy (SDG 7)**: This is the most relevant SDG across the DAX companies, suggesting that these companies are placing a strong emphasis on energy-related issues and clean energy solutions in their operations and reporting.

2. **Industry, Innovation and Infrastructure (SDG 9)**: As the second most relevant SDG, it indicates a significant focus on building resilient infrastructure, promoting inclusive and sustainable industrialization, and fostering innovation among the DAX companies.

3. **Reduced Inequalities (SDG 10)**: Standing as the third, it shows a considerable commitment by the companies to address inequalities within and among countries.

4. **Responsible Consumption and Production (SDG 12)**: This high ranking reflects an emphasis on sustainable management and efficient use of natural resources, as well as a push for more sustainable practices and consumption patterns.

5. **Life Below Water (SDG 14)**: The companies are also paying substantial attention to conserving and sustainably using the oceans, seas, and marine resources, as indicated by its relevance score.

6. **Partnerships for the Goals (SDG 17)**: This indicates the importance placed on multi-stakeholder partnerships that mobilize and share knowledge, expertise, technology, and financial resources to support the achievement of the SDGs in all countries.

7. **Clean Water and Sanitation (SDG 6)** to **Gender Equality (SDG 5)**: The middle-ranked SDGs show moderate to high relevance, suggesting these areas are also recognized within corporate sustainability efforts but with varying degrees of focus.

8. **Life On Land (SDG 15)** and **Peace, Justice and Strong Institutions (SDG 16)**: These SDGs have lower total relevance scores, indicating they are less emphasized across the DAX companies compared to other goals.

9. **Gender Equality (SDG 5)**: As the SDG with the lowest total relevance score, this may suggest that gender equality is not as prominently addressed within the sustainability reports of the DAX companies or is not as aligned with their core business activities as other SDGs.

This interpretation suggests that while there is some focus across the spectrum of SDGs, certain goals related to environmental sustainability, innovation, and infrastructure are given more emphasis in the corporate sustainability reports of DAX companies. Goals related to social issues like gender equality appear to receive less emphasis relative to others.

To gain a more nuanced understanding of why some SDGs score higher than others, it would be beneficial to analyze the content of the reports qualitatively to understand the specific contexts, initiatives, and reporting practices of the DAX companies. Additionally, considering the sectors in which these companies operate could provide further insights into why certain SDGs are more relevant to them.


Conclusion and Recommendations

•	Suggestions for companies, policymakers, and ESG rating providers on how to reduce greenwashing:

To effectively mitigate the issue of greenwashing, companies should adopt a multi-faceted approach encompassing increased transparency, rigorous third-party audits, broad stakeholder engagement, and the strategic implementation of NLP tools. By enhancing the transparency of sustainability reports and providing detailed methodologies behind ESG claims, companies can foster greater accountability. 
Engaging independent third parties to audit these claims and involving a diverse range of stakeholders—such as NGOs, community representatives, and industry experts—in the reporting process further ensures a well-rounded representation of the company’s sustainability efforts. 
Crucially, developing internal NLP tools to continuously analyze and compare their ESG reporting with external media and reports can help companies identify discrepancies and better align their corporate reporting with external perceptions and facts. Policymakers can support these efforts by funding research focused on advanced NLP techniques for monitoring ESG disclosures and promoting regulatory frameworks that mandate the disclosure of both positive and negative sustainability impacts. Similarly, ESG rating providers should integrate these sophisticated NLP methodologies to enhance the analysis of qualitative data in ESG reports, enabling more accurate comparisons with quantitative data and third-party information, thereby ensuring a more reliable assessment of corporate sustainability practices.

